# Introduction to Missing Data 

- [Handling Missing Data](https://jakevdp.github.io/PythonDataScienceHandbook/03.04-missing-values.html)
- [Working with missing data](https://pandas.pydata.org/docs/user_guide/missing_data.html#values-considered-missing)

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

Missing or duplicate data may exist in a data set for a number of different reasons:
- Values are missed during data acquisition process
    - Faulty weather sensors during weather analysis
    - Incomplete patient information for medical diagnosis etc.
- Values deleted accidentally
    - Data loss
    - Mistakenly deleted due to human error
    - Data storage or conversion issues
- Sometimes, missing or duplicate data is introduced as we perform cleaning and transformation tasks such as:
    - Combining data
    - Reindexing data
    - Reshaping data

**Workflow for treating missing values**
1. Convert all missing values to null values.
2. Analyze the amount and type of missingness in the data.
3. Appropriately delete or impute missing values.
4. Evaluate & compare the performance of the treated/imputed dataset.

## Trade-Offs in Missing Data Conventions

There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame. Generally, they revolve around one of two strategies: using a **mask that globally indicates missing values**, or choosing a **sentinel value that indicates a missing entry**.

In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinel value could be some data-specific convention, such as indicating a missing integer value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with **NaN (Not a Number)**, a special value which is part of the IEEE floating-point specification.

None of these approaches is without trade-offs: use of a separate mask array requires allocation of an additional Boolean array, which adds overhead in both storage and computation. **A sentinel value reduces the range of valid values that can be represented, and may require extra (often non-optimized) logic in CPU and GPU arithmetic**. Common special values like NaN are not available for all data types.

As in most cases where no universally optimal choice exists, different languages and systems use different conventions. For example, the R language uses reserved bit patterns within each data type as sentinel values indicating missing data, while the SciDB system uses an extra byte attached to every cell which indicates a NA state.

## None: Pythonic missing data

The first sentinel value used by Pandas is None, a Python singleton object that is often used for missing data in Python code. Because it is a Python object, None cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type 'object' (i.e., arrays of Python objects):

In [ ]:
import numpy as np
import pandas as pd

vals1 = np.array([1, None, 3, 4])
vals1

This dtype=object means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects. While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types:

In [ ]:
for dtype in ["object", "int"]:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

The use of Python objects in an array also means that if you perform aggregations like sum() or min() across an array with a None value, you will generally get an error:

In [ ]:
vals1 = np.array([1, None, 3, 4])
try:
    vals1.sum()
except TypeError as e:
    print(e)

This reflects the fact that addition between an integer and None is undefined.

## NaN: Missing numerical data

The other missing data representation, NaN (acronym for Not a Number), is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [ ]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

Notice that NumPy chose a native floating-point type for this array: this means that unlike the object array from before, this array supports fast operations pushed into compiled code. You should be aware that NaN is a bit like a data virus–it infects any other object it touches. Regardless of the operation, the result of arithmetic with NaN will be another NaN:

In [ ]:
1 + np.nan

In [ ]:
0 * np.nan

Note that this means that aggregates over the values are well defined (i.e., they don't result in an error) but not always useful:

In [ ]:
vals2.sum()

In [ ]:
vals2.min()

In [ ]:
vals2.max()

NumPy does provide some special aggregations that will ignore these missing values:

In [ ]:
np.nansum(vals2)

In [ ]:
np.nanmin(vals2)

In [ ]:
np.nanmax(vals2)

Keep in mind that NaN is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types.

## NaN and None in Pandas

NaN and None both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [ ]:
pd.Series([1, np.nan, 2, None])

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present. For example, if we set a value in an integer array to np.nan, it will automatically be upcast to a floating-point type to accommodate the NA:

In [ ]:
x = pd.Series(range(2), dtype=int)

In [ ]:
x

In [ ]:
x[0] = None

In [ ]:
x

## Missing Data in Pandas

The way in which Pandas handles missing values is constrained by its reliance on the NumPy package, which does not have a built-in notion of NA values for non-floating-point data types.

**pandas uses different sentinel values to represent a missing (also referred to as NA) depending on the data type.**


`numpy.nan` for NumPy data types. The disadvantage of using NumPy data types is that the original data type will be coerced to `np.float64` or `object`.

In [ ]:
pd.Series([1, 2], dtype=np.int64).reindex([0, 1, 2])

In [ ]:
pd.Series([True, False], dtype="bool").reindex([0, 1, 2])

`NaT` for NumPy `np.datetime64`, `np.timedelta64`, and `PeriodDtype`. For typing applications, use api.`types.NaTType`.

In [ ]:
pd.Series([1, 2], dtype=np.dtype("timedelta64[ns]")).reindex([0, 1, 2])

`NA` for StringDtype. These types will maintain the original data type of the data. For typing applications, use `api.types.NAType`.

In [ ]:
pd.Series(["a", "b"], dtype="string").reindex([0, 1, 2])

Equality compaisons between `np.nan`, `NaT`, and `NA` do not act like `None`

In [ ]:
None == None  # noqa: E711

In [ ]:
np.nan == np.nan

In [ ]:
pd.NaT == pd.NaT

In [ ]:
pd.NA == pd.NA

To detect these missing value, use the `isna()` or `notna()` methods. `isna()` or `notna()` will also consider None a missing value.

In [ ]:
pd.isna(pd.Series([1, 2], dtype=np.int64).reindex([0, 1, 2]))

In [ ]:
# isna() or notna() will also consider None a missing value.
ser = pd.Series([1, None], dtype=object)
pd.isna(ser)

## NA semantics

> Experimental: the behaviour of NA` can still change without warning.

Starting from pandas 1.0, an experimental NA value (singleton) is available to represent scalar missing values. The goal of `NA` is provide a “missing” indicator that can be used consistently across data types (instead of np.nan, None or pd.NaT depending on the data type).

For example, when having missing values in a `Series` with the nullable integer dtype, it will use NA:

In [ ]:
s = pd.Series([1, 2, None], dtype="Int64")

In [ ]:
s

In [ ]:
s[2]

In [ ]:
s[2] is pd.NA

Currently, pandas does not yet use those data types using NA by default a DataFrame or Series, so you need to specify the dtype explicitly. 

## Operating on Null Values

There are several useful methods for detecting, removing, and replacing null values in Pandas data structures. They are:

- `isna()`: Generate a boolean mask indicating missing values
- `notna()`: Opposite of isna()
- `dropna()`: Return a filtered version of the data
- `fillna()`: Return a copy of the data with missing values filled or imputed

We will conclude this section with a brief exploration and demonstration of these routines.

### Detecting null values

Pandas data structures have two useful methods for detecting null data: `isna()` and `notna()`. Either one will return a Boolean mask over the data. For example:

In [ ]:
data = pd.Series([1, np.nan, "hello", None])

In [ ]:
data

In [ ]:
data.isna()

As mentioned in Data Indexing and Selection, Boolean masks can be used directly as a Series or DataFrame index:

In [ ]:
data[data.notna()]

The `isna()` and `notna()` methods produce similar Boolean results for DataFrames.

### Dropping null values

In addition to the masking used before, there are the convenience methods, `dropna()` (which removes NA values) and fillna() (which fills in NA values). For a Series, the result is straightforward:


In [ ]:
data.dropna()

For a DataFrame, there are more options. Consider the following DataFrame:

In [ ]:
df = pd.DataFrame([[1, np.nan, 2], [2, 3, 5], [np.nan, 4, 6]])

In [ ]:
df

We cannot drop single values from a DataFrame; we can only drop full rows or full columns. Depending on the application, you might want one or the other, so dropna() gives a number of options for a DataFrame.

By default, dropna() will drop all rows in which any null value is present:

In [ ]:
df.dropna(axis="index")

Alternatively, you can drop NA values along a different axis; axis=1 drops all columns containing a null value:

In [ ]:
df.dropna(axis="columns")

But this drops some good data as well; you might rather be interested in dropping rows or columns with all NA values, or a majority of NA values. This can be specified through the how or thresh parameters, which allow fine control of the number of nulls to allow through.

The default is how='any', such that any row or column (depending on the axis keyword) containing a null value will be dropped. You can also specify how='all', which will only drop rows/columns that are all null values:

In [ ]:
df[3] = np.nan

In [ ]:
df

In [ ]:
df.dropna(axis="columns", how="all")

For finer-grained control, the thresh parameter lets you specify a minimum number of non-null values for the row/column to be kept:

In [ ]:
df.dropna(axis="index", thresh=3)

Here the first and last row have been dropped, because they contain only two non-null values.

In [ ]:
# Get percentage of missing values in each column
(df.isna().sum(axis="index") / df.shape[0]) * 100

In [ ]:
# Remove the columns with more than 50% missing values
df = pd.DataFrame([[1, np.nan, 2], [2, np.nan, 5], [np.nan, 4, 6], [7, np.nan, 9], [np.nan, 10, 11], [12, np.nan, 14], [15, np.nan, 17]])
df

In [ ]:
df.dropna(axis="columns", thresh=int(df.shape[0] * 0.5))

### Filling null values

Sometimes rather than dropping NA values, you'd rather replace them with a valid value. This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values. You could do this in-place using the `isna()` method as a mask, but because it is such a common operation Pandas provides the `fillna()` method, which returns a copy of the array with the null values replaced.

Consider the following Series:

In [ ]:
data = pd.Series([1, np.nan, 2, None, 3], index=list("abcde"))

In [ ]:
data

We can fill NA entries with a single value, such as zero:

In [ ]:
data.fillna(0)

We can specify a forward-fill to propagate the previous value forward:

In [ ]:
# forward-fill
data.ffill()

Or we can specify a back-fill to propagate the next values backward:

In [ ]:
# back-fill
data.bfill()

For DataFrames, the options are similar, but we can also specify an axis along which the fills take place:

In [ ]:
df

In [ ]:
df.ffill(axis="columns")

Notice that if a previous value is not available during a forward fill, the NA value remains.

In [ ]:
data = {"A": [1, 2, None, 4], "B": [None, 2, 3, 4]}
df = pd.DataFrame(data)
df

In [ ]:
# Fill with column mean
df_filled = df.fillna(df.mean())
df_filled

In [ ]:
# Fill with median
df_filled = df.fillna(df.median())
df_filled

## Identifying Hidden Missing Data

In [ ]:
data_path = Path.cwd().parent / "data" / "wh_2015_special.csv"
happiness2015 = pd.read_csv(data_path)
happiness2015.head()

In [ ]:
happiness2015.isna().sum()

In [ ]:
# some missing values in the "Region" column has a dot instead of NaN
happiness2015["Region"].unique()

In [ ]:
happiness2015[happiness2015["Region"] == "."]

In [ ]:
happiness2015 = pd.read_csv(data_path, na_values=".")
happiness2015.isna().sum()

In [ ]:
happiness2015[happiness2015["Region"].isna()]

In [ ]:
happiness2015 = happiness2015.replace("?", np.nan)
happiness2015[happiness2015["Region"].isna()]

## Advanced Visualization of Missing Data

In [ ]:
data_path = Path.cwd().parent / "data" / "pima-indians-diabetes_data.csv"
diabetes = pd.read_csv(data_path)
diabetes.head()

In [ ]:
diabetes.info()

In [ ]:
diabetes.describe()

<p>The <code>diabetes</code> DataFrame has 0's in the column <code>BMI</code>. But <code>BMI</code> cannot be 0. It should instead be <code>NaN</code>.

In [ ]:
# Set the 0 values of column 'BMI' to np.nan
diabetes.loc[diabetes["BMI"] == 0, "BMI"] = np.nan

In [ ]:
diabetes.describe()

In [ ]:
# Analyzing missingness percentage
diabetes_nullity = (diabetes.isna().sum() / diabetes.shape[0]) * 100
diabetes_nullity

<p>You'll use the <code>misssingno</code> package which is built for visualizing missing values.

[missingno GitHub](https://github.com/ResidentMario/missingno)

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt

In [ ]:
# Visualize the missingness summary
msno.matrix(diabetes)

# Display nullity matrix
plt.show()

The msno.matrix nullity matrix is a data-dense display which lets you quickly visually pick out patterns in data completion.

The sparkline at right summarizes the general shape of the data completeness and points out the rows with the maximum and minimum nullity in the dataset.

Finding correlations between missing data helps you gain a deeper understanding of the type of missing data as well as provides suitable ways in which the missing values can be addressed. 

In [ ]:
# Plot missingness heatmap of diabetes
msno.heatmap(diabetes)
# Show plot
plt.show()

In [ ]:
# Sort diabetes dataframe on 'Serum Insulin'
sorted_values = diabetes.sort_values("Serum_Insulin")

# Visualize the missingness summary of sorted
msno.matrix(sorted_values)

# Display nullity matrix
plt.show()

In [ ]:
# Drop all rows where 'Glucose' has a missing value
diabetes = diabetes.dropna(subset=["Glucose"], how="all")

In [ ]:
# Drop rows where 'BMI' has a missing value
diabetes = diabetes.dropna(subset=["BMI"], how="all")

In [ ]:
# Visualize the missingness in the data
msno.matrix(diabetes)
plt.show()